# Chapter 11 PRACTICAL METHODOLOGY

* 유주원

# Chapter 11.5 Debugging Strategies

* machine learning 성능이 저하되어 나타날 때, 우리는 이 원인이 알고리즘 본연의 성능인지 아니면 어떠한 버그를 가지고 있는 것인지 쉽게 판별하기가 어렵다.
* 특히 machine learning 같은 경우는 각각의 요소가 adaptive하기 때문에, 어떤 한 파트가 부러졌다고 하더라도 다른 부분에서 이를 보완할 수 있고 여전히 수용 가능한 성능을 낼 수가 있다.
* 예를 들어, 아래와 같은 bias 계산 과정에서 오류가 발생했다고 가정하자. α는 학습 속도를 말한다.<br>
<center>$ b <= b - α$</center>
* 위의 식에서 bias는 학습 동안에 계속적으로 음수를 취하게 되며, 이는 합리적인 학습 알고리즘이 아니다.
* weight는 bias의 음수에 대한 보상을 위해 적절한 값을 취하게 될 것이며, 단지 모델의 결과 값만을 통해서 해당 모델의 버그 여부를 판별해서는 안될 것이다.
* 이러한 버그를 판별하기 위해 우리는 올바은 행동을 실제로 예측할 수 있는 간단한 케이스를 설계하거나, neural net의 일부분을 개별적으로 실험할 수 있는 테스트를 설계한다.
* debugging test를 진행하기 위해서는 아래의 몇 가지 중요한 사실이 포함되어야 한다.

#### <font color="FF5555">실행 중인 모델을 시각화하라</font>
* 이미지 내에서 객체를 찾기 위한 모델을 training 할때, 모델이 검출한 결과를 직접 확인해라.
* 혹은 음성을 생성하는 모델을 훈련시킬 때, 모델이 생성한 음성 결과를 직접 들어라.
* 당연한 말이지만, 이를 통해 정확도 같은 정량적인 측정만을 보는 관행을 벗어날 수가 있다.
* evaluation bug는 가장 치명적인 버그가 될 수 있다. <br>왜냐하면 시스템이 잘 동작하지 않음에도 불구하고 잘 동작한다고 믿게 만들 수 있기 때문이다.

#### <font color="FF5555">최악의 실수를 시각화하라</font>
* 대부분의 모델은 수행 중인 task에 대한 측정 값을 출력할 수가 있다.
* 예를 들어 softmax 분류기는 각각의 class를 확률로 변환하여 출력 할 수가 있다.
* 그런데 전형적으로 maximum likelyhood training 결과는 정확한 예측에 대한 확률 보다 오히려 과대 평가된 값을 결과로 도출하기는 하지만 오답에 대해서는 더 적은 확률을 결과로 내놓기 때문에 유용하다.
* 모델이 맞추기 가장 어려운 데이터 예제를 직접 확인함으로써, data의 전처리 혹은 label 방법등에 대한 문제를 발견할 수가 있다.
* 한 예로, Street view를 문자로 옮기는 시스템 내에서 주소를 검출하는 시스템에서 문제가 발생했는데 이미지를 너무 타이트하게 자르거나, 자리수를 생략하는 동작이 발견되었다.
* 해당 시스템에서는 이러한 이미지들에 대한 올바른 답에 대한 확률을 낮추었고, 이미지들을 정렬해서 가장 확실한 실수를 발견했다.
* cropping system에 문제가 있는 것이 확인이 되었고, 이미지를 더 넓게 crop하도록 수정함으로써 성능을 올렸다.

#### <font color="FF5555">train 또는 test error를 통해 software를 추론해라.</font>
* 만약 train error가 낮지만 test error가 높다면, 모델이 overfitting 되었다는 것을 확인할 수가 있다.
* overfitting 이외의 가능성으로 훈련된 모델이 다시 로드될 때 문제가 생겨서 error가 높게 나왔다고 생각할 수가 있다.
* 만약에 train error와 test error가 둘 다 높다면, software의 결함인지 아니면 모델이 underfitting 되었는지를 구분 짓는 것이 어렵다.
* 이러한 경우에는 더 많은 test가 진행되어야 한다.
 
#### <font color="FF5555">작은 data set에 맞춰라.</font>
* 만약 training set에서 높은 에러가 발생했다면, underfitting 때문인지 아니면 software 결함인지를 결정해라.
* 이를 확인하기 위해 작은 data set에 맞는 작은 모델에 시작해라
* 예를 들어 하나의 출력만 가지고 있는 분류 데이터의 경우에는 biases 설정 만으로도 모델을 만들 수가 있다.
* 만약 해당 데이터에 대한 훈련이 힘들다면, autoencoder를 통해 단일 예제를 재생성하거나 generative model을 이용해서 데이터를 resampling 함으로써 데이터를 생성할 수가 있다.
 
#### <font color="FF5555">back-propagated derivative를 확인해라.</font>
* back-propagated derivative를 유한 차분법에 의해 계산된 derivative와 비교해라.
* 유한 차분법 : 연속된 공간에서 정의된 미분 방정식을 차분 방정식으로 근사시켜 푸는 방법
<img src="./figures/fig1.png" />
* 우리는 벡터 기반의 Jacobian 또는 gradient에 대한 test를 원하지만, 유한 차분법에 의해서는 단일 도함수밖에 얻을 수가 없다. 
* 유한 차분법을 mn 번 실행시킴으로써 g의 모든 부분 도함수를 도출하거나, g의 input과 output에 random projection을 적용해 볼 수도 있다.
* random projection : 차원 축소 기법 중에 하나. PCA보다는 부정확하나 수가 많은 경우 적은 계산 비용으로 효율적인 차원 감소가 가능하다.
* u와 v가 각각 랜덤 벡터로 선택된 f(x) = $u^{t}g(vx)$를 가지고 예를 들어 보자.
* g가 올바르게 back-propagation이 되어야 $f^{'}(x)$를 계산할 수 있다. 또한 f는 단일 input과 output을 가지기 때문에 유한 차분법을 이용해서 효율적으로 구할 수가 있다.
* 복소수를 수치 계산해야 하는 경우라면, 함수의 입력으로 복소수를 사용함으로써 gradient를 측정하는 방법이 있다. (Squire and Trapp, 1998)
<img src="./figures/fig2.png" />
* 서로 다른 점에서 f 값의 차이를 취하리 때문에 cancellation effect(간섭 효과??)가 없다.

#### <font color="FF5555">activations과 gradient의 histogram을 모니터하라.</font>
* 많은 양의 training iteration을 통해 수집된 neural network activation과 gradient의 통계를 시각화 하는 것은 매우 유용하다.
* hidden unit의 pre-activation value는 이 유닛이 saturate 되었는지, 얼마나 동작하는지 등에 대해 말해준다.
* 예를 들어, rectifiers 같은 경우 얼마나 자주 꺼져 있는지? 항상 꺼져 있는 unit이 있는지 등을 말해주고, tanh 같은 경우에는 pre-activations의 절대값의 평균은 unit이 얼마나 saturate 되었는지를 말해준다. 
* parameter의 magnitude와 parameter gradient의 magnitude를 비교하는 것은 상당히 유용하다.
* Bottou(2015)는 parameter magnitude의 update가 parameter magnitude의 50%나 0.001% 정도가 아니라 1% 정도로 나타나야 된다고 제시하고 있다.
* 또한 데이터가 sparse할 경우(natural language) 몇몇의 파라미터는 거의 update가 되지 않을 수 있으며, 모니터링 시에는 이러한 사항들을 명심해야 한다.
